# GMM Test Data Mockup Notebook

This notebook will produce the test files for GMM dashboard, all output files are stored in tsv format.

For more information please refer to https://github.com/WEHI-ResearchComputing/Genomics-Metadata-Multiplexing/tree/shiny-r-dev/test.

In [1]:
from operations import *

## Operation 1: Generate Sample Sheet based on given Plate Layout Template

In [8]:
def plate_to_samplesheet(xlsx_file):
    '''
    Convert plate layout spreadsheet to samplesheet.
    '''
    wb = openpyxl.load_workbook(xlsx_file)

    # iterate through all sheets
    full_samplesheet = pd.DataFrame()
    for sheet in wb:
        if not sheet.title.lower().startswith('lce'):
            print('Skipping sheet {}'.format(sheet.title))
            continue

        sample_start_cell = find_sample_start_cell(sheet)
        assert sample_start_cell, 'Could not find sample start cell in sheet {}'.format(sheet.title)

        well_start_cell = find_well_start(sheet)
        assert well_start_cell, 'Could not find well start cell in sheet {}'.format(sheet.title)

        sample_lookup = get_sample_lookup(sheet, sample_start_cell)
        sample_list = get_sample_list(sheet, sample_lookup, well_start_cell)
        samplesheet = pd.DataFrame(sample_list, columns=['well_position', 'sample'])
        samplesheet['plate'] = sheet.title

        full_samplesheet = pd.concat([full_samplesheet, samplesheet])

    # reorder columns
    full_samplesheet = full_samplesheet[['plate', 'well_position', 'sample']]
    return full_samplesheet

In [9]:
# generate sample sheet from based on provided plate_spreadsheet_template.xlsx
sample_sheet_df = plate_to_samplesheet('./plate_spreadsheet_template.xlsx')

(
    sample_sheet_df.head(),
    sample_sheet_df.to_csv('op1.plate_layout_to_spreadsheet.tsv', sep='\t', index=False)
)

(    plate well_position            sample
 0  LCE123            A1  Test before sort
 1  LCE123            A2   Test after sort
 2  LCE123            A3           SampleA
 3  LCE123            A4           SampleA
 4  LCE123            A5           SampleA,
 None)

## Operation 2: Collate Multiple FCS files into a single FCS file

In [11]:
def collate_fcs_files(fcs_files, upload_dir):
    '''
    Collate FCS files into a single dataframe, adding columns for plate and sample name.
    '''
    fcs_data = pd.DataFrame()

    for fcs_file in fcs_files:
        # fcs_savepath = os.path.join(upload_dir, fcs_file.filename)
        fcs_savepath = fcs_file  # alter this function for mock up test result purpose
        meta, data = fcsparser.parse(fcs_savepath, meta_data_only=False, reformat_meta=True)
        data = data.sort_values('Time')
        
        plate, sample = get_plate_and_sample_from_filepath(fcs_savepath)
        data['plate'] = plate
        data['well_position'] = get_well_positions(meta)
        data['sample'] = sample

        fcs_data = pd.concat([fcs_data, data])

    return fcs_data

In [12]:
# collate multiple fcs files into a single tsv output
collated_fcs_df = collate_fcs_files(['./14Jun23_INX_NKC_084_LCE662.fcs'], "")  # provide a list of fcs files

# mock up plate data for LCE123
collated_fcs_df['plate'] = 'LCE123'
collated_fcs_df['sample'] = sample_sheet_df['sample'][:len(collated_fcs_df)]

(
    collated_fcs_df.head(),
    collated_fcs_df.to_csv('./op2.collate_fcs_files.tsv', sep='\t', index=False)
)

(           FSC-A    FSC-H          SSC-A    SSC-H    CD16 FITC       CD56 PE  \
 0  115089.296875  69562.0   75108.632812  44791.0   129.710007  37108.621094   
 1  127671.296875  83696.0   64815.761719  43674.0   112.270004  17709.240234   
 2  102016.796875  74176.0   50605.429688  35155.0   105.730003  25967.160156   
 3  123290.093750  71681.0  100057.640625  55735.0   553.720032     66.299995   
 4  110112.296875  68001.0   86295.304688  55742.0  1148.859985    149.940002   
 
          DAPI         Time   plate well_position            sample  
 0   61.410000  1698.400024  LCE123            A3  Test before sort  
 1   52.509998  1865.300049  LCE123            A4   Test after sort  
 2   41.829998  2052.899902  LCE123            A5           SampleA  
 3  203.809998  2330.300049  LCE123            A6           SampleA  
 4   97.900002  2758.300049  LCE123            A7           SampleA  ,
 None)

## Operation 3: Merge Data with Sample Sheet and Template Sheet

In [59]:
def load_excel_samplesheet(template_sheet_filepath):
    df = pd.read_excel(template_sheet_filepath, skiprows=1, header=None)

    def combine_columns(a, b):
        if pd.isna(a):
            return b
        elif pd.isna(b):
            return a
        return f'{a}_{b}'
        
    new_columns = [combine_columns(a, b) for a, b in zip(df.iloc[0], df.iloc[1])]

    df.columns = new_columns
    df = df.drop(index=[0, 1])

    df.rename({'Plate#': 'plate', 'Well position': 'well_position', 'Sample name': 'sample'}, axis=1, inplace=True)    
    return df

In [82]:
def merge_data_with_samplesheet(spreadsheet_filepath, fcs_file, template_sheet_filepath):
    '''
    Merges processed/uploaded data (may be sample sheet and/or fcs data).
    Will merge into a template if provided.
    '''
    fcs_data = pd.read_csv(fcs_file, sep='\t') if fcs_file else None
    is_xlsx = spreadsheet_filepath.endswith('.xlsx')
    merged_data = pd.DataFrame()

    if template_sheet_filepath and is_xlsx:
        raise Exception('Cannot merge template sheet with xlsx file. Please upload a tsv file with plate, sample and well positions.')
    elif template_sheet_filepath:
       
        template = load_excel_samplesheet(template_sheet_filepath)
        spreadsheet = pd.read_csv(spreadsheet_filepath, sep='\t')
        # spreadsheet.rename({'Plate#': 'plate', 'Well position': 'well_position', 'Sample name': 'sample'}, axis=1, inplace=True)

        for plate in spreadsheet.plate.unique():
            plate_data = template.copy()
            plate_data['plate'] = plate
            plate_data = pd.merge(plate_data, spreadsheet,
                                  left_on=['plate', 'well_position', 'sample'],
                                  right_on=['plate', 'well_position', 'sample'], how='left')

            merged_data = pd.concat([merged_data, plate_data])
        
    elif not is_xlsx:
        merged_data = pd.read_csv(spreadsheet_filepath, sep='\t')

    if fcs_data is None:
        return merged_data
    
    if is_xlsx:
        spreadsheet = load_excel_samplesheet(spreadsheet_filepath)
        samples_colname = [col for col in spreadsheet.columns if col.lower() == 'sample' or col.lower() == 'sample name'][0]

        return pd.merge(spreadsheet, fcs_data, 
                        left_on=['Plate#', 'Well position', samples_colname],
                        right_on=['plate', 'well_position', 'sample'], how='left')
    else:
        return pd.merge(merged_data, fcs_data, on=['plate', 'sample', 'well_position'], how='left')

In [83]:
merged_samplesheet_fcs_and_template_sheet_df = merge_data_with_samplesheet('./op1.plate_layout_to_spreadsheet.tsv', 
                                                                           './op2.collate_fcs_files.tsv', 
                                                                           './template_sheet.xlsx')

(
    merged_samplesheet_fcs_and_template_sheet_df.head(30), 
    merged_samplesheet_fcs_and_template_sheet_df.to_csv('op3.merged_sample_sheet.tsv', index=False, sep='\t')
)

(     plate well_position Sample type\n(SC or MB) Tissue type\n(if required)  \
 0   LCE123            A1                   empty                        NaN   
 1   LCE123            A2                   empty                        NaN   
 2   LCE123            A3                     NaN                        NaN   
 3   LCE123            A4                     NaN                        NaN   
 4   LCE123            A5                     NaN                        NaN   
 5   LCE123            A6                     NaN                        NaN   
 6   LCE123            A7                     NaN                        NaN   
 7   LCE123            A8                     NaN                        NaN   
 8   LCE123            A9                     NaN                        NaN   
 9   LCE123           A10                     NaN                        NaN   
 10  LCE123           A11                     NaN                        NaN   
 11  LCE123           A12               

## Operation 4: Merge Primer Index File with Operation 3 Result (if provide Primer Index File)

In [86]:
# load primer index template
primer_index_df = pd.read_excel('primer_index_template.xlsx', sheet_name='Sample primer & index', skiprows=3)

# mockup primer index data by setting plate# to LCE123, mockup sample data based on operation 1
primer_index_df['Plate#'] = 'LCE123'
primer_index_df['Sample name'] = sample_sheet_df['sample']

# generate mockup test_primer_index.tsv file
primer_index_df.to_csv('test_primer_index.tsv', sep='\t', index=False)

primer_index_df.rename({'Plate#': 'plate', 'Well position': 'well_position', 'Sample name': 'sample'}, axis=1, inplace=True)    
# generate mockup test result file
merged_primer_index_df = pd.merge(merged_samplesheet_fcs_and_template_sheet_df, primer_index_df, 
                                  left_on=['plate', 'well_position', 'sample'], 
                                  right_on=['plate', 'well_position', 'sample'],
                                  suffixes=('', '_primer'))

(
    merged_primer_index_df.head(),
    merged_primer_index_df.to_csv('op4.merged_primer_index.tsv', sep='\t', index=False)
)

(    plate well_position Sample type\n(SC or MB) Tissue type\n(if required)  \
 0  LCE123            A1                   empty                        NaN   
 1  LCE123            A2                   empty                        NaN   
 2  LCE123            A3                     NaN                        NaN   
 3  LCE123            A4                     NaN                        NaN   
 4  LCE123            A5                     NaN                        NaN   
 
              sample FACs gate\n(if required) C-RT1-_Primer name  \
 0  Test before sort                      NaN            removed   
 1   Test after sort                      NaN            removed   
 2           SampleA                      NaN                 99   
 3           SampleA                      NaN                100   
 4           SampleA                      NaN                101   
 
   RD1 index (cell index)_index sequence \n(as in C-RT1-primer)  \
 0                                            r